In [1]:
import re
import io
import time
import numpy as np
import pandas as pd
import boto3

In [2]:
session = boto3.Session(profile_name='insight')
athena_client = session.client('athena')
s3_client = session.client('s3')

In [3]:
def wait_for_query_completion(client, query_execution_id: str):
    """      
    determines if the query has finished excuting. If it has not, the function waits for the query to finished.
    Parameters:
        query_execution_id: the query execution id (as a string)
        client: an aws client or session.client
        returns the location of query results (string)
    """
    max_execution = 600
    state = 'RUNNING'
    while (max_execution > 0 and state in ['RUNNING', 'QUEUED']):
        max_execution = max_execution - 1
        response = client.get_query_execution(QueryExecutionId = query_execution_id)
        if 'QueryExecution' in response and 'Status' in response['QueryExecution'] and 'State' in response['QueryExecution']['Status']:
            state = response['QueryExecution']['Status']['State']
            if state == 'FAILED':
                return state
            elif state == 'SUCCEEDED':
                s3_path = response['QueryExecution']['ResultConfiguration']['OutputLocation']
                return s3_path
        time.sleep(1)
    return 'FAILED'

In [4]:
context = {
    'Database': 'genes_and_health',
    'Catalog': 'AwsDataCatalog'
}
local_query_string = 'SELECT * FROM qt_gene_based LIMIT 50;'

In [5]:
%%time
response = athena_client.start_query_execution(QueryString=local_query_string, QueryExecutionContext=context)
query_execution_id = response['QueryExecutionId']
#print('Execution ID: ', query_execution_id)
s3_path = wait_for_query_completion(athena_client, query_execution_id)

CPU times: user 18.9 ms, sys: 2.79 ms, total: 21.7 ms
Wall time: 3.23 s


In [6]:
s3_path

's3://aws-athena-query-results-638341270768-us-east-1/ef58b7b7-2621-4a8e-9293-d74213e802ce.csv'

In [7]:
file_name = re.findall('.*\/(.*)', s3_path)[0]
file_location = s3_path.replace('s3://','').split('/', 1)[0]
print('file name: ', file_name, 'file location: ', file_location)

file name:  ef58b7b7-2621-4a8e-9293-d74213e802ce.csv file location:  aws-athena-query-results-638341270768-us-east-1


In [8]:
s3_client = session.client('s3')
obj = s3_client.get_object(Bucket=file_location, Key=file_name)
result_df = pd.read_csv(io.BytesIO(obj['Body'].read()), header=0)
result_df

,chrom,genpos,id,mask,a1freq,n,test,beta,se,chisq,...,gene id,variant mask,freq,raw path,phenotype,stat,gene-based,female-only,male-only,label
0,1,925926,SAMD11(ENSG00000187634).MASK_B.singleton,B.singleton,0.003036,1153,ADD,0.363867,0.353264,1.060920,...,ENSG00000187634,B,singleton,tak-insight-open-geneshealthconsort-silv/raw/2...,AFP,max,True,False,False,AFP (burden)
1,1,925926,SAMD11(ENSG00000187634).MASK_B.0.0001,B.0.0001,0.003036,1153,ADD,0.363867,0.353264,1.060920,...,ENSG00000187634,B,0.0001,tak-insight-open-geneshealthconsort-silv/raw/2...,AFP,max,True,False,False,AFP (burden)
2,1,925926,SAMD11(ENSG00000187634).MASK_B.0.001,B.0.001,NaN,1153,ADD-SKAT,NaN,NaN,0.074253,...,ENSG00000187634,B,0.001,tak-insight-open-geneshealthconsort-silv/raw/2...,AFP,max,True,False,False,AFP (burden)
3,1,925926,SAMD11(ENSG00000187634).MASK_B.0.001,B.0.001,NaN,1153,ADD-SKATO,NaN,NaN,0.074253,...,ENSG00000187634,B,0.001,tak-insight-open-geneshealthconsort-silv/raw/2...,AFP,max,True,False,False,AFP (burden)
4,1,925926,SAMD11(ENSG00000187634).MASK_B.0.001,B.0.001,0.003903,1153,ADD,0.085238,0.312806,0.074253,...,ENSG00000187634,B,0.001,tak-insight-open-geneshealthconsort-silv/raw/2...,AFP,max,True,False,False,AFP (burden)
5,1,925926,SAMD11(ENSG00000187634).MASK_B.0.01,B.0.01,0.009974,1153,ADD,-0.055619,0.187779,0.087732,...,ENSG00000187634,B,0.01,tak-insight-open-geneshealthconsort-silv/raw/2...,AFP,max,True,False,False,AFP (burden)
6,1,925926,SAMD11(ENSG00000187634).MASK_C.singleton,C.singleton,0.007372,1153,ADD,0.328905,0.227895,2.082920,...,ENSG00000187634,C,singleton,tak-insight-open-geneshealthconsort-silv/raw/2...,AFP,max,True,False,False,AFP (burden)
7,1,925926,SAMD11(ENSG00000187634).MASK_C.0.0001,C.0.0001,0.007372,1153,ADD,0.328905,0.227895,2.082920,...,ENSG00000187634,C,0.0001,tak-insight-open-geneshealthconsort-silv/raw/2...,AFP,max,True,False,False,AFP (burden)
8,1,925926,SAMD11(ENSG00000187634).MASK_C.0.001,C.0.001,NaN,1153,ADD-SKAT,NaN,NaN,0.646439,...,ENSG00000187634,C,0.001,tak-insight-open-geneshealthconsort-silv/raw/2...,AFP,max,True,False,False,AFP (burden)
9,1,925926,SAMD11(ENSG00000187634).MASK_C.0.001,C.0.001,NaN,1153,ADD-SKATO,NaN,NaN,0.646439,...,ENSG00000187634,C,0.001,tak-insight-open-geneshealthconsort-silv/raw/2...,AFP,max,True,False,False,AFP (burden)


## Query for all results for a given gene (TYK2)

In [10]:
queryStringGene = 'SELECT * FROM qt_gene_based WHERE qt_gene_based."gene id"=\'ENSG00000105397\';'

In [11]:
%%time
response = athena_client.start_query_execution(QueryString=queryStringGene, QueryExecutionContext=context)
query_execution_id = response['QueryExecutionId']
s3_path = wait_for_query_completion(athena_client, query_execution_id)
if not s3_path == 'FAILED':
    file_name = re.findall('.*\/(.*)', s3_path)[0]
    file_location = s3_path.replace('s3://','').split('/', 1)[0]
    obj = s3_client.get_object(Bucket=file_location, Key=file_name)
    result_df = pd.read_csv(io.BytesIO(obj['Body'].read()), header=0)

CPU times: user 616 ms, sys: 26.6 ms, total: 643 ms
Wall time: 3min 20s


In [12]:
result_df

,chrom,genpos,id,mask,a1freq,n,test,beta,se,chisq,...,gene id,variant mask,freq,raw path,phenotype,stat,gene-based,female-only,male-only,label
0,19,10350840,TYK2(ENSG00000105397).MASK_B.singleton,B.singleton,0.006505,1153,ADD,-0.279340,0.251489,1.233760,...,ENSG00000105397,B,singleton,tak-insight-open-geneshealthconsort-silv/raw/2...,AFP,max,True,False,False,AFP (burden)
1,19,10350840,TYK2(ENSG00000105397).MASK_B.0.0001,B.0.0001,0.006505,1153,ADD,-0.279340,0.251489,1.233760,...,ENSG00000105397,B,0.0001,tak-insight-open-geneshealthconsort-silv/raw/2...,AFP,max,True,False,False,AFP (burden)
2,19,10350840,TYK2(ENSG00000105397).MASK_B.0.001,B.0.001,NaN,1153,ADD-SKAT,NaN,NaN,0.245660,...,ENSG00000105397,B,0.001,tak-insight-open-geneshealthconsort-silv/raw/2...,AFP,max,True,False,False,AFP (burden)
3,19,10350840,TYK2(ENSG00000105397).MASK_B.0.001,B.0.001,NaN,1153,ADD-SKATO,NaN,NaN,0.245660,...,ENSG00000105397,B,0.001,tak-insight-open-geneshealthconsort-silv/raw/2...,AFP,max,True,False,False,AFP (burden)
4,19,10350840,TYK2(ENSG00000105397).MASK_B.0.001,B.0.001,0.008239,1153,ADD,-0.110291,0.222521,0.245660,...,ENSG00000105397,B,0.001,tak-insight-open-geneshealthconsort-silv/raw/2...,AFP,max,True,False,False,AFP (burden)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7303,19,10350840,TYK2(ENSG00000105397).MASK_D.0.0001,D.0.0001,0.002020,15843,ADD,0.018873,0.120692,0.024452,...,ENSG00000105397,D,0.0001,tak-insight-open-geneshealthconsort-silv/raw/2...,Waist_circumference,min,True,False,False,Waist circumference (burden)
7304,19,10350840,TYK2(ENSG00000105397).MASK_D.0.001,D.0.001,NaN,15843,ADD-SKAT,NaN,NaN,0.409387,...,ENSG00000105397,D,0.001,tak-insight-open-geneshealthconsort-silv/raw/2...,Waist_circumference,min,True,False,False,Waist circumference (burden)
7305,19,10350840,TYK2(ENSG00000105397).MASK_D.0.001,D.0.001,NaN,15843,ADD-SKATO,NaN,NaN,0.227166,...,ENSG00000105397,D,0.001,tak-insight-open-geneshealthconsort-silv/raw/2...,Waist_circumference,min,True,False,False,Waist circumference (burden)
7306,19,10350840,TYK2(ENSG00000105397).MASK_D.0.001,D.0.001,0.008647,15843,ADD,-0.041688,0.057690,0.522176,...,ENSG00000105397,D,0.001,tak-insight-open-geneshealthconsort-silv/raw/2...,Waist_circumference,min,True,False,False,Waist circumference (burden)


## Query for all signigicant burden associations for a given gene (SERPINA1)

In [5]:
queryStringGene = 'SELECT * FROM qt_gene_based WHERE qt_gene_based."gene id"=\'ENSG00000197249\' AND qt_gene_based.significant = true;'

In [6]:
%%time
response = athena_client.start_query_execution(QueryString=queryStringGene, QueryExecutionContext=context)
query_execution_id = response['QueryExecutionId']
s3_path = wait_for_query_completion(athena_client, query_execution_id)
if not s3_path == 'FAILED':
    file_name = re.findall('.*\/(.*)', s3_path)[0]
    file_location = s3_path.replace('s3://','').split('/', 1)[0]
    obj = s3_client.get_object(Bucket=file_location, Key=file_name)
    result_df = pd.read_csv(io.BytesIO(obj['Body'].read()), header=0)

CPU times: user 390 ms, sys: 15.4 ms, total: 405 ms
Wall time: 3min 30s


In [7]:
result_df

,chrom,genpos,id,mask,a1freq,n,test,beta,se,chisq,...,gene id,variant mask,freq,raw path,phenotype,stat,gene-based,female-only,male-only,label


## Query for all single variant associations within the genomic region of a given gene (TYK2)

In [8]:
queryStringGene = 'SELECT * FROM qt_single_variant WHERE qt_single_variant.chrom = 19 AND qt_single_variant.genpos < 10380608 AND qt_single_variant.genpos > 10350533;'

In [9]:
%%time
response = athena_client.start_query_execution(QueryString=queryStringGene, QueryExecutionContext=context)
query_execution_id = response['QueryExecutionId']
s3_path = wait_for_query_completion(athena_client, query_execution_id)
if not s3_path == 'FAILED':
    file_name = re.findall('.*\/(.*)', s3_path)[0]
    file_location = s3_path.replace('s3://','').split('/', 1)[0]
    obj = s3_client.get_object(Bucket=file_location, Key=file_name)
    result_df = pd.read_csv(io.BytesIO(obj['Body'].read()), header=0)

CPU times: user 1.11 s, sys: 62.9 ms, total: 1.17 s
Wall time: 8min 40s


In [10]:
result_df

,chrom,genpos,id,allele0,allele1,a1freq,n,test,beta,se,chisq,log10p,significant,raw path,phenotype,stat,gene-based,female-only,male-only,label
0,19,10350843,chr19_10350843_G_A,G,A,0.002098,13584,ADD,0.008030,0.125962,0.004064,0.022655,False,tak-insight-open-geneshealthconsort-silv/raw/2...,AST,max,False,False,False,AST
1,19,10350910,chr19_10350910_T_C,T,C,0.017005,13584,ADD,-0.037337,0.044208,0.713312,0.399739,False,tak-insight-open-geneshealthconsort-silv/raw/2...,AST,max,False,False,False,AST
2,19,10351069,chr19_10351069_C_T,C,T,0.000221,13584,ADD,-0.029673,0.387217,0.005872,0.027373,False,tak-insight-open-geneshealthconsort-silv/raw/2...,AST,max,False,False,False,AST
3,19,10351070,chr19_10351070_G_A,G,A,0.001031,13584,ADD,0.062226,0.179512,0.120158,0.137354,False,tak-insight-open-geneshealthconsort-silv/raw/2...,AST,max,False,False,False,AST
4,19,10351109,chr19_10351109_G_C,G,C,0.000994,13584,ADD,-0.206436,0.176301,1.371080,0.616854,False,tak-insight-open-geneshealthconsort-silv/raw/2...,AST,max,False,False,False,AST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40063,19,10378350,chr19_10378350_G_A,G,A,0.000726,15843,ADD,-0.437253,0.192884,5.138910,1.630870,False,tak-insight-open-geneshealthconsort-silv/raw/2...,Waist_circumference,min,False,False,False,Waist circumference
40064,19,10378363,chr19_10378363_A_G,A,G,0.005144,15843,ADD,0.054369,0.075027,0.525128,0.329140,False,tak-insight-open-geneshealthconsort-silv/raw/2...,Waist_circumference,min,False,False,False,Waist circumference
40065,19,10378365,chr19_10378365_G_A,G,A,0.001168,15843,ADD,0.098673,0.158914,0.385544,0.271930,False,tak-insight-open-geneshealthconsort-silv/raw/2...,Waist_circumference,min,False,False,False,Waist circumference
40066,19,10378371,chr19_10378371_A_C,A,C,0.000379,15843,ADD,-0.205617,0.257894,0.635678,0.371325,False,tak-insight-open-geneshealthconsort-silv/raw/2...,Waist_circumference,min,False,False,False,Waist circumference


In [11]:
athena_client.close()
s3_client.close()